In [1]:
import pandas as pd

In [2]:
#Read CSV file
df = pd.read_csv(r"C:\Users\foooo\Desktop\Gaming_scrapping\100\file\null.csv")

In [3]:
df.shape

(4767, 13)

In [4]:
#df = df.iloc[3178:4768]

In [5]:
# drop the columns will make duplcated in columns after fill null
df.drop(columns=['OS','Processor','Memory','Graphics','DirectX','Storage','Additional Notes'],axis=1,inplace=True)

In [6]:
#df = df.head(10)

In [7]:
import requests
from bs4 import BeautifulSoup

In [ ]:


#creat list for data 
extracted_data = []

for url in df['Links']:
    try:
        response = requests.get(url)
        response.raise_for_status()  

        soup = BeautifulSoup(response.text, 'html.parser')

        full_text = soup.get_text(separator=" ", strip=True) 

        extracted_data.append({'url': url, 'extracted_text': full_text})

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        extracted_data.append({'url': url, 'extracted_text': 'Error'})

extracted_df = pd.DataFrame(extracted_data)

print(extracted_df.head())


In [ ]:
extracted_df[['before_os', 'Requirements']] = extracted_df['extracted_text'].str.extract(r'(.*?)(?:OS \*:)(.*)', expand=True)


print(extracted_df)

In [ ]:
import re

In [ ]:
def extract_system_requirements(text):
    requirements = {
        'OS': '', 'Processor': '', 'Memory': '', 'Graphics': '',
        'DirectX': '', 'Storage': '', 'Additional Notes': ''
    }
    
    # Find the system requirements section
    match = re.search(r'(Minimum|Recommended):\s*(.*?)(?=(Recommended:|$))', text, re.DOTALL)
    if not match:
        return requirements

    req_text = match.group(2)
    
    # Extract each requirement
    for key in requirements.keys():
        pattern = rf'{key}:?\s*(.*?)(?=(?:{"|".join(requirements.keys())})|$)'
        match = re.search(pattern, req_text, re.IGNORECASE | re.DOTALL)
        if match:
            requirements[key] = match.group(1).strip()
    
    # If Additional Notes is empty, use any remaining text
    if not requirements['Additional Notes']:
        remaining = re.sub(r'.?Storage:.?\n', '', req_text, flags=re.DOTALL).strip()
        requirements['Additional Notes'] = remaining
    
    return requirements

# Assuming your DataFrame is named df
extracted_df[['OS', 'Processor', 'Memory', 'Graphics', 'DirectX', 'Storage', 'Additional Notes']] = extracted_df['extracted_text'].apply(lambda x: pd.Series(extract_system_requirements(x)))

# Display the first few rows of the processed DataFrame
print(extracted_df[['OS', 'Processor', 'Memory', 'Graphics', 'DirectX', 'Storage', 'Additional Notes']].head())



In [ ]:
extracted_df

In [ ]:
extracted_df.to_csv('df_fix_null3.csv')